In [ ]:
//====================================================================================================
//The Free Edition of C++ to Python Converter limits conversion output to 100 lines per snippet.

//To purchase the Premium Edition, visit our website:
//https://www.tangiblesoftwaresolutions.com/order/order-cplus-to-python.html
//====================================================================================================

import math



class Globals:
    #C++ TO PYTHON CONVERTER WARNING: The following #include directive was ignored:
    ##include <Rcpp.h>
    #C++ TO PYTHON CONVERTER WARNING: The following #include directive was ignored:
    ##include <R.h>
    #extern "C" __declspec(dllexport)
    #  
    # function beta_iteration for updating the beta. 
    # x is the predictor matrix, y is the reponse viable. 
    # beta1 is the coefficients for updating.
    # j is the subscript of the excluded column of x
    # tau is the quantile parameter.
    # return the vector prebeta for caculating the weighted median 
    # 
    #extern "C"{
    #function for iteration to get the optimal solution. 
    # y is the responses, x is the predictors, beta1 is the parameters, nyrow is the row length 
    # of y, nxcol is the column length of x, tau is the quantile parameter.
    # void F77_NAME(xssort)(double*, double*, int*, int*);
#C++ TO PYTHON CONVERTER TODO TASK: The implementation of the following method could not be found:
    #xssort_(UnnamedParameter, UnnamedParameter2, UnnamedParameter3, UnnamedParameter4)

    # C WRAPPER FUNCTION FOR xssort SUBROUTINE
    @staticmethod
    def quicksort(x, w, n):
        kflag = 2
        #  CALLING FORTRAN	FUNCTION FROM C
        # F77_CALL(xssort)(x, w, n, &kflag)
        temp_ref_kflag = RefObject(kflag);
        Globals.xssort_(x, w, n, temp_ref_kflag)
        kflag = temp_ref_kflag.arg_value

    @staticmethod
#C++ TO PYTHON CONVERTER TODO TASK: Pointer arithmetic is detected on the parameter 'x':
#ORIGINAL LINE: void QCD(double *x, double *beta, double *intval, double *penweight, double *residuals, int *n, int *p, int *intercept, double *tau, double *eps, int *maxin)
    def QCD(x, beta, intval, penweight, residuals, n, p, intercept, tau, eps, maxin):
        # x is the design matrix (just covariates, no column of ones); n x p matrix (converted to vector)
        # beta is the initial (and will be the final) value of coefficients; vector of length p
        # intval is the initial (and will be the final) value of the intercept; can be anything if no intercept in model
        # penweight is the penalty weight for each coefficient; vector of length p
        # residuals is the vector of residuals from current value of betas; vector of length n ( = y - x%*%beta )
        # n is the number of observations
        # p is the number of coefficients (not including intercept)
        # intercept is the intercept indicator, 1 is intercept, 0 is no intercept
        # tau is the quantile of interest
        # thresh is the convergence threshold
        # maxin is maximum number of iterations
        iter = 0 # Iteration count
        col = None # Column index (p columns in x and then do intercept last)
        row = None # Row index, (row, col) entry of x, keep track of nonzero x entries for each column
        rowcol = None
        nonzerox = None

        weight = [0 for _ in range(n.arg_value + 1)] # Weight vector
        length_weight = None # Length of weight vector

        pre_value_final = [0 for _ in range(n.arg_value+1)] # pre_value_final is the vector to save the weighted median vector

        # Used for computing weighted median
        count3 = None
        weight_sum = None
        temp1 = None

        # Keep track whether beta vector has converged
        betavecDiff = None
        betaDiff = None
        newBeta = None # placeholder for new beta


        while (iter < maxin.arg_value): #number of inside iterations
            col = 0
            betavecDiff = 0
            while col < p[0]: # This is the loop for the coefficients, do the intercept last
                temp1 = 0 # Used for calculation of weighted median
                weight_sum = 0 # sum of the weight vector
                row = 0
                nonzerox = -1
                rowcol = n.arg_value*col
                while row < n.arg_value: # loop through the entries in x of column number col
                    if x[rowcol] != 0:
                        nonzerox += 1
                        pre_value_final[nonzerox] = residuals[row]

                        if pre_value_final[nonzerox] > 0:
                            weight[nonzerox] = abs(x[rowcol])* tau.arg_value
                        else:
                            weight[nonzerox] = abs(x[rowcol])*(1 - tau.arg_value)

                        pre_value_final[nonzerox] = (pre_value_final[nonzerox] + x[rowcol]*beta[col])/(x[rowcol])
                        if *tau.arg_value >= 0.5:
                            pre_value_final[nonzerox] = -(pre_value_final[nonzerox])

                        weight_sum += weight[nonzerox]

                    row += 1
                    rowcol += 1

                # Compute the extra pseudo-observation (penalty for the coefficient)
                nonzerox += 1
                pre_value_final[nonzerox] = 0

                weight[nonzerox] = penweight[col]
                weight_sum += weight[nonzerox]

                # Compute the weighted median
                length_weight = nonzerox+1
                temp_ref_pre_value_final = RefObject(pre_value_final);
                temp_ref_weight = RefObject(weight);
                temp_ref_length_weight = RefObject(length_weight);
                Globals.quicksort(temp_ref_pre_value_final, temp_ref_weight, temp_ref_length_weight)
                length_weight = temp_ref_length_weight.arg_value
                weight = temp_ref_weight.arg_value
                pre_value_final = temp_ref_pre_value_final.arg_value
                count3 = -1
                temp1 = 0
                weight_sum = weight_sum/2 # Don't need sum anymore, just need sum/2
                while temp1 < weight_sum:
                    count3 += 1
                    temp1 = temp1 + weight[count3]

                # Update the coefficient
                newBeta = pre_value_final[count3]
                if *tau.arg_value >= 0.5:
                    newBeta = -newBeta

                betaDiff = beta[col] - newBeta
                betavecDiff = betavecDiff + betaDiff *betaDiff
                beta[col] = newBeta

                # Update the residual vector if newBeta has changed
                if betaDiff != 0:
                    row = 0
                    rowcol = n.arg_value*col
                    while row < n.arg_value:
                        residuals[row] = residuals[row] + x[rowcol]*betaDiff
                        row += 1
                        rowcol += 1

                col += 1 # Next coefficient and repeat

            if *intercept.arg_value == 1: # If intercept, update the intercept
                temp1 = 0 # Used for calculation of univariate tauth quantile
                weight_sum = 0 # sum of the weight vector (for the intercept, everything has weight of 1)
                row = 0
                nonzerox = -1
                while row < n.arg_value: # loop through the entries in x of column number col
                    nonzerox += 1
                    pre_value_final[nonzerox] = residuals[row] + intval[0]
                    weight[nonzerox] = 1
                    row += 1
                    weight_sum = weight_sum + weight[nonzerox]

                # Compute the univariate tauth quantile
                length_weight = nonzerox+1
                temp_ref_pre_value_final2 = RefObject(pre_value_final);
                temp_ref_weight2 = RefObject(weight);
                temp_ref_length_weight2 = RefObject(length_weight);
                Globals.quicksort(temp_ref_pre_value_final2, temp_ref_weight2, temp_ref_length_weight2)
                length_weight = temp_ref_length_weight2.arg_value
                weight = temp_ref_weight2.arg_value
                pre_value_final = temp_ref_pre_value_final2.arg_value
                count3 = -1
                temp1 = 0

//====================================================================================================
//End of the allowed output for the Free Edition of C++ to Python Converter.

//To purchase the Premium Edition, visit our website:
//https://www.tangiblesoftwaresolutions.com/order/order-cplus-to-python.html
//====================================================================================================


#Helper class added by C++ to Python Converter:

# ----------------------------------------------------------------------------------------
#	Copyright © 2021 - 2022 Tangible Software Solutions, Inc.
#	This class can be used by anyone provided that the copyright notice remains intact.
#
#	This class is used to replicate the ability to pass arguments by reference in Python.
# ----------------------------------------------------------------------------------------
class RefObject:
    def __init__(self, ref_arg):
        self.arg_value = ref_arg
